## Inicialización

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from google.colab import drive

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras import layers

from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD

random_state = 33

In [ ]:
#Para acceder a los ficheros de Google Drive
drive.mount('/content/drive')
# La carpeta datos debe estar en vuestro Drive, dentro de la carpeta 'Colab Notebooks'

##Función Pasar de vector codificado a matriz de adyacencia

In [ ]:
def convertir_vector_matriz(caso, N):
  vector = caso[:int(N*(N-1)/2)]
  #Inicializar la matriz de adyacencia
  matriz_ad = [[False for _ in range(N)] for _ in range(N)]
  index = 0  # Índice para recorrer el vector
  for i in range(N):
      for j in range(i + 1, N):  # Empezar desde i+1 para omitir la diagonal
          if vector[index]:  # Si el valor en el vector es True
              matriz_ad[i][j] = True
              matriz_ad[j][i] = True  # Rellenar el valor simétrico para la matriz no dirigida
          index += 1
  return matriz_ad

#Experimento Vertex Cover $N=20$ y $10^5$ casos

Debido al alto coste computacional de entrenar las redes neuronales, ejecutar etse experimento lleva una cantidad considerable de horas.

El problema utiliza $N$ objetos exactamente y los datos se cargan desde un fichero .csv como un vector de X casos y cada posición es un vector de $4N+2$ variables que representan:   

- 0 : Número de vértices
- $1$ - $(\frac{(N(N-1))}{2})$   : Mitad triangular superior de la matriz de adyacencia.

- $(\frac{(N(N-1))}{2}+1)$  -  $(\frac{(N(N-1))}{2} + N+1)$ : Vector binario solucion peor.  
- $(\frac{(N(N-1))}{2} + N+1)$  -  $(\frac{(N(N-1))}{2} + 2N+1)$ : Vector binario de solucion mejor.



### Cargar Datos

In [ ]:
file_path = 'drive/MyDrive/Colab Notebooks/datos/TFG/data_E1_VertexCover.csv'

datos_originales = np.genfromtxt(file_path, delimiter=",") #Funcion cargar datos desde txt, y en este caso desde csv


Comprobación y eliminación de Duplicados en los datos

In [ ]:
# Comprobando duplicados en datos
unique_rows, indices = np.unique(datos_originales, axis=0, return_index=True)
num_duplicates = datos_originales.shape[0] - unique_rows.shape[0]

print("Número de filas duplicadas en los datos:", num_duplicates)

# Eliminar duplicados
datos = datos_originales[np.sort(indices)]


unique_rows, indices = np.unique(datos, axis=0, return_index=True)
num_duplicates = datos.shape[0] - unique_rows.shape[0]

print("Número de filas duplicadas en los datos despúes de eliminar duplicados:", num_duplicates)

In [ ]:
numItems = datos[0][0]
tamMatriz = numItems*(numItems-1)/2
x = datos[:, 1: int(tamMatriz+numItems+1)]  # datos de entrada
y = datos[:, int(tamMatriz+numItems+1):]   # datos de etiqueta
x = np.array(x, np.float64)
y = np.array(y, np.float64)

print("Número de objetos (N) =", numItems)
# Mostrar dimension del conjunto de muestras total
print("Forma de vector X de muestras:", x.shape)
print("Forma de vector Y de etiquetas:", y.shape)
print("Forma de vector datos:", datos.shape)

print("Ejemplos:")
print("X = ",x[0])
print("Y = ",y[0])

### Separación de datos en Train y Test

Separo el conjunto de casos en test y train para evaluar los modelos utilizados. También se pueden utilizar funciones como `sklearn.model_selection.train_test_split()` pero de esta forma se pueden guardar los índices para realizar comprobaciones de forma más sencilla.

In [ ]:
#Separate data in test and train

np.random.seed(33)
trainPortion = 0.8 #porcentaje de train, el porcentaje de test será la resta de 1 menos el porcentaje de train

#-------------Obtener índices

indexesData = np.arange(len(y)) #Indices del conjunto de muestras
#-------------Desordenar indices y separar en rain y test

np.random.shuffle(indexesData) #Desordenar indices de las muestras
numberTrain = round(len(indexesData)*trainPortion) #numero de muestras para train
trainIndexes = indexesData[:numberTrain]
testIndexes = indexesData[numberTrain:]

#-------------Datos desorden:ados para train y test

trainX = x[trainIndexes]
testX = x[testIndexes]
trainY = y[trainIndexes]
testY = y[testIndexes]

#-------------Copia de los datos para mantener datos originales después del preprocesado

original_trainX = trainX.copy()
original_testX = testX.copy()
original_trainY = trainY.copy()
original_testY = testY.copy()

#-------------Mostrar resultados

print('Muestras totales:  {}'.format(len(trainY)+len(testX)))
print('Muestras train:  {}'.format(len(trainY)))
print('Muestras test:  {}'.format(len(testX)))


## Preprocesamiento de los datos

Como los datos han sido generados artificialmente y no se han obtenido de fuentes externas, no es necesario realizar la búsqueda de datos faltantes o erroneos ni desbalanceo en las proporciones ya que los datos han sido generados teniendo en cuenta estos factores. La codificación de los datos también se ha tenido en cuenta en al generación de los datos.

### Comprobación de Duplicados en los datos

In [ ]:
# Comprobando duplicados en datos
unique_rows= np.unique(datos, axis=0)
num_duplicates = datos.shape[0] - unique_rows.shape[0]

print("Número de filas duplicadas en los datos:", num_duplicates)

### Escalar Datos

En este problema no hace falta escalar los datos

## Funciones y Metrica fitness

In [ ]:
def valor_fitness(instancia, caso):
  instancia_local = instancia.copy()
  N = len(instancia_local)
  sin_cubrir = convertir_vector_matriz(caso,N)
  #sin_cubrir = copy.deepcopy(matriz_ad)

  # Iterar sobre la solución inicial y marcar las aristas cubiertas
  for i, cubierto in enumerate(instancia_local):
      if cubierto:
          for j in range(N):  # Modificar sin usar sintaxis de NumPy
                sin_cubrir[j][i] = False
                sin_cubrir[i][j] = False

  # Verificar aristas restantes y añadir vértices a la solución si es necesario
  for i in range(N):
      for j in range(i + 1, N):
          if sin_cubrir[i][j]:  # Si la arista entre i y j no está cubierta
              instancia_local[i] = True
              for k in range(N):  # Modificar sin usar sintaxis de NumPy
                  sin_cubrir[k][i] = False
                  sin_cubrir[i][k] = False
  return np.sum(instancia_local)

def metrica_valor_fitness_entrada(y_true, y_pred, x):
  mejora = 0
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    #valor_salida_real = np.sum(y_true)
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    valor_entrada = np.sum(x[i][int(num_items*(num_items-1)/2):])
    mejora += valor_salida_pred / valor_entrada
  return mejora/num_tot

def metrica_valor_fitness_salida(y_true, y_pred, x):
  mejora = 0
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valor_salida_real = np.sum(y_true[i])
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    #valor_entrada = np.sum(x[i][int(num_items*(num_items-1)/2):])
    if(valor_salida_real == 0):
      mejora += valor_salida_pred
    else:  mejora += valor_salida_pred / valor_salida_real
  return mejora/num_tot


def metrica_porcentaje_mejor_entrada(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    #valor_salida_real = np.sum(y_true)
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    valor_entrada = np.sum(x[i][int(num_items*(num_items-1)/2):])
    if valor_salida_pred < valor_entrada:
      num_aciertos += 1
  return num_aciertos/num_tot


def metrica_porcentaje_mejor_salida(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valor_salida_real = np.sum(y_true[i])
    #print(y_true,valor_salida_real)
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    #valor_salida = np.sum(x[i][int(num_items*(num_items-1)/2):])
    if valor_salida_pred < valor_salida_real:
      num_aciertos += 1
  return num_aciertos/num_tot


def metrica_porcentaje_mejor_igual_entrada(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    #valor_salida_real = np.sum(y_true)
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    valor_entrada = np.sum(x[i][int(num_items*(num_items-1)/2):])
    if valor_salida_pred <= valor_entrada:
      num_aciertos += 1
  return num_aciertos/num_tot

def metrica_porcentaje_mejor_igual_salida(y_true, y_pred, x):
  num_aciertos = 0
  num_tot = len(x)
  num_items = len(y_pred[0])
  umbral = 0.5
  predictions_binary = (y_pred >= umbral).astype(int)
  for i in range(num_tot):
    valor_salida_real = np.sum(y_true[i])
    #print(y_true,valor_salida_real)
    valor_salida_pred = valor_fitness(y_pred[i],x[i])
    #valor_salida = np.sum(x[i][int(num_items*(num_items-1)/2):])
    if valor_salida_pred <= valor_salida_real:
      num_aciertos += 1
  return num_aciertos/num_tot



In [ ]:
print("Ratio de mejora de las soluciones etiquetas respecto de las soluciones iniciales:",metrica_valor_fitness_entrada(trainY,trainY,trainX))
print("Porcentaje de soluciones etiquetas mejores estricto que las soluciones iniciales:",metrica_porcentaje_mejor_entrada(trainY,trainY,trainX)*100,"%")
print("Porcentaje de soluciones etiquetas mejores o iguales que las soluciones iniciales:",metrica_porcentaje_mejor_igual_entrada(trainY,trainY,trainX)*100,"%")

## Creación del Modelo

Valores de entrada y salida de los modelos:

In [ ]:
input_shape  = tamMatriz + numItems
output_shape = numItems  #  Numero de valores a predecir (la solucion al problema)

Vamos a implementar una función que nos cree el modelo que vamos a entrenar y evaluar.

In [ ]:
def create_model(dense_neurons, input_shape, output_shape):
    local_model = keras.Sequential()

    # Capa de entrada
    local_model.add(layers.Input(shape=(int(input_shape),)))

    # capas ocultas con activación Relu
    for neurons in dense_neurons:
      local_model.add(layers.Dense(neurons, activation='relu'))

    # Capa de salida para clasificación con tantas neuronas como salidas esperadas
    local_model.add(layers.Dense(output_shape, activation='sigmoid'))
    return local_model

## Grid Search

Definimos la función para utilizar grid_search

In [ ]:
def single_grid_search(dense_neurons, input_shape, output_shape, X, Y, epochs, batch_size, learn_rate, _n_splits = 5, _random_state = 33):

    # Inicializar KFold
    kfold = KFold(n_splits=_n_splits, shuffle=True, random_state=_random_state)

    accuracy_results = []
    loss_results = []
    mejora_entrada_results = []
    mejora_salida_results = []
    porc_mejor_entrada_results = []
    porc_mejor_salida_results = []
    porc_mejor_igual_entrada_results = []
    porc_mejor_igual_salida_results = []

    # Iterar sobre cada fold (validación cruzada)
    for train_index, test_index in kfold.split(X):
        # Dividir datos en entrenamiento y prueba
        trainX, valX = X[train_index], X[test_index]
        trainY, valY = Y[train_index], Y[test_index]

        # Crear modelo
        local_model = create_model(dense_neurons, input_shape, output_shape)

        # Compilar el modelo
        sgd_optimizer = SGD(learning_rate=learn_rate)
        local_model.compile(optimizer=sgd_optimizer,
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

        # Entrenar el modelo
        local_model.fit(
            trainX, trainY,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
        )

        # Evaluar el modelo
        scores = local_model.evaluate(valX, valY, verbose=0)
        loss_results.append(scores[0])
        accuracy_results.append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = local_model.predict(valX,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results.append(metrica_valor_fitness_entrada(valY,y_pred_binary,valX))
        mejora_salida_results.append(metrica_valor_fitness_salida(valY,y_pred_binary,valX))
        porc_mejor_entrada_results.append(metrica_porcentaje_mejor_entrada(valY,y_pred_binary,valX))
        porc_mejor_salida_results.append(metrica_porcentaje_mejor_salida(valY,y_pred_binary,valX))
        porc_mejor_igual_entrada_results.append(metrica_porcentaje_mejor_igual_entrada(valY, y_pred_binary, valX))
        porc_mejor_igual_salida_results.append(metrica_porcentaje_mejor_igual_salida(valY, y_pred_binary, valX))

    # Devolver el promedio de las evaluaciones
    return np.mean(loss_results), np.mean(accuracy_results),np.mean(mejora_entrada_results),np.mean(mejora_salida_results),np.mean(porc_mejor_entrada_results),np.mean(porc_mejor_salida_results), np.mean(porc_mejor_igual_entrada_results), np.mean(porc_mejor_igual_salida_results), loss_results

Valores de los hiperparámetros a probar:

In [ ]:
# Valores de los hiperparámetros para el modelo base
dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
epochs = 100
learning_rate = 0.1
batch_size = 64
k_folds = 2

# Valores de los  distintos hiperparámetros
learning_rate_search  = [0.1,0.01,0.001,0.0001,0.00001]
batch_size_search = [16,32,64,96,128]

> Hiperparámetro de la tasa de aprendizaje (learning rate)

In [ ]:
accuracy_mean_table = []
loss_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
porc_mejor_igual_entrada_table = []
porc_mejor_igual_salida_table = []

for lr in learning_rate_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX, trainY, epochs, batch_size, lr, k_folds, random_state)

    loss_mean_table.append(results[0])
    accuracy_mean_table.append(results[1])
    mejora_entrada_table.append(results[2])
    mejora_salida_table.append(results[3])
    porc_mejor_entrada_table.append(results[4])
    porc_mejor_salida_table.append(results[5])
    porc_mejor_igual_entrada_table.append(results[6])
    porc_mejor_igual_salida_table.append(results[7])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tasa de Aprendizaje': learning_rate_search,'Error de validación(función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores estricto que Inicial':porc_mejor_entrada_table,'Porc Mejores o iguales que Inicial':porc_mejor_igual_entrada_table,
                           'Porc Mejores que Etiqueta':porc_mejor_salida_table, 'Porc Mejores o iguales que Etiqueta':porc_mejor_igual_salida_table})
display(tableFrame)

> Hiperparámetro del tamaño de lote (batch size)

In [ ]:
accuracy_mean_table = []
loss_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
porc_mejor_igual_entrada_table = []
porc_mejor_igual_salida_table = []

for bs in batch_size_search:
    # Entrenar y evaluar
    results = single_grid_search(dense_neurons, input_shape, output_shape, trainX, trainY, epochs, bs, learning_rate, k_folds, random_state)

    loss_mean_table.append(results[0])
    accuracy_mean_table.append(results[1])
    mejora_entrada_table.append(results[2])
    mejora_salida_table.append(results[3])
    porc_mejor_entrada_table.append(results[4])
    porc_mejor_salida_table.append(results[5])
    porc_mejor_igual_entrada_table.append(results[6])
    porc_mejor_igual_salida_table.append(results[7])


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Tamaño de Lote': batch_size_search,'Error de validación(función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores estricto que Inicial':porc_mejor_entrada_table,'Porc Mejores o iguales que Inicial':porc_mejor_igual_entrada_table,
                           'Porc Mejores que Etiqueta':porc_mejor_salida_table, 'Porc Mejores o iguales que Etiqueta':porc_mejor_igual_salida_table})
display(tableFrame)

## Modelo Final

In [ ]:
# Valores de los hiperparámetros para el modelo final
dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
batch_size = 16
epochs = 200
learning_rate = 0.1


final_model = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
final_model.compile(optimizer=sgd_optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

# Entrenar el modelo
final_model.fit(
    trainX, trainY,
    epochs=epochs,
    batch_size=batch_size,
    verbose=2,
)

## Resultados

In [ ]:
accuracy_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
porc_mejor_igual_entrada_table = []
porc_mejor_igual_salida_table = []
loss_mean_table = []

# Evaluar el modelo
scores = final_model.evaluate(testX, testY, verbose=0)
loss_mean_table.append(scores[0])
accuracy_mean_table.append(scores[1])

# Evaluar metricas personalizadas
y_pred = final_model.predict(testX,verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)

mejora_entrada_table.append(metrica_valor_fitness_entrada(testY, y_pred_binary, testX))
mejora_salida_table.append(metrica_valor_fitness_salida(testY, y_pred_binary, testX))
porc_mejor_entrada_table.append(metrica_porcentaje_mejor_entrada(testY, y_pred_binary, testX))
porc_mejor_salida_table.append(metrica_porcentaje_mejor_salida(testY, y_pred_binary, testX))
porc_mejor_igual_entrada_table.append(metrica_porcentaje_mejor_igual_entrada(testY, y_pred_binary, testX))
porc_mejor_igual_salida_table.append(metrica_porcentaje_mejor_igual_salida(testY, y_pred_binary, testX))


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Error de test (función de pérdida)':loss_mean_table,'Accuracy':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores estricto que Inicial':porc_mejor_entrada_table,'Porc Mejores o igual que Inicial':porc_mejor_igual_entrada_table,
                           'Porc Mejores que Etiqueta':porc_mejor_salida_table, 'Porc Mejores o igual que Etiqueta':porc_mejor_igual_salida_table})
display(tableFrame)

## Curvas de Aprendizaje

In [ ]:
def create_learning_curves_mejora(curves_model, train_X, train_Y, val_X, val_Y, batch_size,epochs, iters):
    accuracy_results = [[],[]]
    loss_results = [[],[]]
    mejora_entrada_results = [[],[]]
    mejora_salida_results = [[],[]]
    porc_mejor_entrada_results = [[],[]]
    porc_mejor_salida_results = [[],[]]
    porc_mejor_igual_entrada_results = [[],[]]
    porc_mejor_igual_salida_results = [[],[]]

    for i in range(iters):
        # Entrenar el modelo
        curves_model.fit(
            train_X, train_Y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=2,
        )

        e_type = 0 # Entrenameinto
        # Evaluar el modelo
        scores = curves_model.evaluate(train_X, train_Y, verbose=0)
        loss_results[e_type].append(scores[0])
        accuracy_results[e_type].append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(train_X,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results[e_type].append(metrica_valor_fitness_entrada(train_Y,y_pred_binary,train_X))
        mejora_salida_results[e_type].append(metrica_valor_fitness_salida(train_Y,y_pred_binary,train_X))
        porc_mejor_entrada_results[e_type].append(metrica_porcentaje_mejor_entrada(train_Y,y_pred_binary,train_X))
        porc_mejor_salida_results[e_type].append(metrica_porcentaje_mejor_salida(train_Y,y_pred_binary,train_X))
        porc_mejor_igual_entrada_results[e_type].append(metrica_porcentaje_mejor_igual_entrada(train_Y,y_pred_binary,train_X))
        porc_mejor_igual_salida_results[e_type].append(metrica_porcentaje_mejor_igual_salida(train_Y,y_pred_binary,train_X))


        e_type = 1 # Validación
        # Evaluar el modelo
        scores = curves_model.evaluate(val_X, val_Y, verbose=0)
        loss_results[e_type].append(scores[0])
        accuracy_results[e_type].append(scores[1])

        # Evaluar metricas personalizadas
        y_pred = curves_model.predict(val_X,verbose=0)
        umbral = 0.5
        y_pred_binary = (y_pred >= umbral).astype(int)

        mejora_entrada_results[e_type].append(metrica_valor_fitness_entrada(val_Y,y_pred_binary,val_X))
        mejora_salida_results[e_type].append(metrica_valor_fitness_salida(val_Y,y_pred_binary,val_X))
        porc_mejor_entrada_results[e_type].append(metrica_porcentaje_mejor_entrada(val_Y,y_pred_binary,val_X))
        porc_mejor_salida_results[e_type].append(metrica_porcentaje_mejor_salida(val_Y,y_pred_binary,val_X))
        porc_mejor_igual_entrada_results[e_type].append(metrica_porcentaje_mejor_igual_entrada(val_Y,y_pred_binary,val_X))
        porc_mejor_igual_salida_results[e_type].append(metrica_porcentaje_mejor_igual_salida(val_Y,y_pred_binary,val_X))

    return accuracy_results, loss_results, mejora_entrada_results, mejora_salida_results, porc_mejor_entrada_results, porc_mejor_salida_results, porc_mejor_igual_entrada_results, porc_mejor_igual_salida_results


dense_neurons  = (numItems*numItems,numItems*numItems,numItems*numItems/2,numItems*2)
batch_size = 16
learning_rate = 0.1
curves_model = create_model(dense_neurons, input_shape, output_shape)

# Compilar el modelo
sgd_optimizer = SGD(learning_rate=learning_rate)
curves_model.compile(optimizer=sgd_optimizer,
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
epochs = 5
iters = 20
curves_results = create_learning_curves_mejora(curves_model, trainX, trainY, testX,testY, batch_size, epochs,iters)

In [ ]:
accuracy_mean_table = []
mejora_entrada_table = []
mejora_salida_table = []
porc_mejor_entrada_table = []
porc_mejor_salida_table = []
loss_mean_table = []

# Evaluar el modelo
scores = curves_model.evaluate(testX, testY, verbose=0)
loss_mean_table.append(scores[0])
accuracy_mean_table.append(scores[1])

# Evaluar metricas personalizadas
y_pred = curves_model.predict(testX,verbose=0)

umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
mejora_entrada_table.append(metrica_valor_fitness_entrada(testY,y_pred_binary,testX))
mejora_salida_table.append(metrica_valor_fitness_salida(testY,y_pred_binary,testX))
porc_mejor_entrada_table.append(metrica_porcentaje_mejor_entrada(testY,y_pred_binary,testX))
porc_mejor_salida_table.append(metrica_porcentaje_mejor_salida(testY,y_pred_binary,testX))


#Crear DataFrame y mostrarlo
tableFrame = pd.DataFrame({'Error de validación medio (función de pérdida)':loss_mean_table,'Accuracy medio':accuracy_mean_table,'Mejora Solucion Incial':mejora_entrada_table,
                           'Mejora Solucion Etiqueta':mejora_salida_table,'Porc Mejores que Inicial':porc_mejor_entrada_table, 'Porc Mejores que Etiqueta':porc_mejor_salida_table})
display(tableFrame)

### Ratio de mejora soluciones Iniciales

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Iniciales",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[2][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ratio de mejora soluciones Etiquetas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje de la Mejora respecto de las Soluciones Etiquetas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Ratio de Mejora",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[3][0], 'b',label=r'$Mejora_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), curves_results[3][1], 'orange',label=r'$Mejora_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Porcentaje de soluciones mejores que Iniciales

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje del Porcentaje de Mejores soluciones respecto de las Iniciales",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Porcentaje de Mejores soluciones",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[4][0])*100, 'b',label=r'$Porcentaje_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[4][1])*100, 'orange',label=r'$Porcentaje_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Porcentaje de soluciones mejores que Etiquetas

In [ ]:
plt.figure(figsize=(12, 7))
plt.title("Curva de Aprendizaje del Porcentaje de Mejores soluciones respecto de las Etiquetas",fontsize = 15)
plt.xlabel("Época",fontsize = 15)
plt.ylabel("Porcentaje de Mejores soluciones",fontsize = 15)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[5][0])*100, 'b',label=r'$Porcentaje_{in}$', linewidth=2)
plt.plot(range(epochs, epochs*(iters+1),epochs), np.array(curves_results[5][1])*100, 'orange',label=r'$Porcentaje_{out}$', linewidth=2)

plt.legend(fontsize = 15)
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

## Algoritmo Genetico


### Funciones de adaptación

In [ ]:
import copy
def convertir_vector_matriz(caso, N):
    vector = caso[:int(N*(N-1)/2)]
    matriz_ad = [[False for _ in range(N)] for _ in range(N)]
    index = 0
    for i in range(N):
        for j in range(i + 1, N):
            if vector[index]:
                matriz_ad[i][j] = True
                matriz_ad[j][i] = True
            index += 1
    return matriz_ad

def concatenar_solucion_instancia(instancia,sol):
    vector_transformado = []
    for aux in instancia:
        vector_transformado.append(aux)

    for aux in sol:
        vector_transformado.append(aux)

    return vector_transformado


def transformar_soluciones_no_factibles(instancia, sol):
  sol_local = sol.copy()
  N = len(sol_local)
  matriz_ad = convertir_vector_matriz(instancia,N)
  sin_cubrir = copy.deepcopy(matriz_ad)

  # Iterar sobre la solución inicial y marcar las aristas cubiertas
  for i, cubierto in enumerate(sol_local):
      if cubierto:
          for j in range(N):  # Modificar sin usar sintaxis de NumPy
                sin_cubrir[j][i] = False
                sin_cubrir[i][j] = False

  # Verificar aristas restantes y añadir vértices a la solución si es necesario
  for i in range(N):
      for j in range(i + 1, N):
          if sin_cubrir[i][j]:  # Si la arista entre i y j no está cubierta
              sol_local[i] = True
              for k in range(N):  # Modificar sin usar sintaxis de NumPy
                  sin_cubrir[k][i] = False
                  sin_cubrir[i][k] = False
  return sol_local


### Algoritmo genético simple

In [ ]:
import random
import numpy as np
from random import getrandbits, randint

def valor_fitness(instancia_local, caso):
    N = len(instancia_local)
    sin_cubrir = convertir_vector_matriz(caso, N)
    valor_fit = np.sum(instancia_local)

    for i, cubierto in enumerate(instancia_local):
        if cubierto:
            for j in range(N):
                sin_cubrir[j][i] = False
                sin_cubrir[i][j] = False

    for i in range(N):
        for j in range(i + 1, N):
            if not instancia_local[i] and not instancia_local[j] and sin_cubrir[i][j]:
                valor_fit += N

    return valor_fit

def media_fitness(poblacion, caso):
    sum_tot = sum(valor_fitness(x, caso) for x in poblacion if valor_fitness(x, caso) != float('inf'))
    num_validas = len([x for x in poblacion if valor_fitness(x, caso) != float('inf')])
    return sum_tot / num_validas if num_validas > 0 else float('inf')

def min_fitness(poblacion, caso):
    valores_fitness = [valor_fitness(x, caso) for x in poblacion if valor_fitness(x, caso) != float('inf')]
    return min(valores_fitness) if valores_fitness else float('inf')

def generar_sol(num_vertices):
    return [getrandbits(1) for _ in range(num_vertices)]

def generar_pobl(num_sols, num_vertices):
    return [generar_sol(num_vertices) for _ in range(num_sols)]

def elegir_antecesor(valores, fitness_total, indice_a_ignorar):
    acumulado = 0
    valor_random = random.random()

    if indice_a_ignorar != -1:
        fitness_total -= valores[0][indice_a_ignorar]

    for indice, i in enumerate(valores[0]):
        if indice_a_ignorar == indice or i == float('inf'):
            continue
        acumulado += i
        if acumulado / fitness_total >= valor_random:
            return indice

def cruzar(antecesor1, antecesor2, tipo='un_punto'):
    if tipo == 'un_punto':
        punto = random.randint(1, len(antecesor1) - 1)
        hijo1 = antecesor1[:punto] + antecesor2[punto:]
        hijo2 = antecesor2[:punto] + antecesor1[punto:]
    elif tipo == 'dos_puntos':
        punto1 = random.randint(1, len(antecesor1) - 2)
        punto2 = random.randint(punto1 + 1, len(antecesor1) - 1)
        hijo1 = antecesor1[:punto1] + antecesor2[punto1:punto2] + antecesor1[punto2:]
        hijo2 = antecesor2[:punto1] + antecesor1[punto1:punto2] + antecesor2[punto2:]
    return hijo1, hijo2

def op_genetico(poblacion, caso, num_sols_total, tipo_cruce='un_punto', mut_prob=0.05):
    soluciones = [[valor_fitness(x, caso), x] for x in poblacion]
    soluciones.sort(key=lambda x: x[0])

    nuevas_sols = []
    while len(nuevas_sols) < num_sols_total:
        antecesor1, antecesor2 = cruzar(soluciones[0][1], soluciones[1][1], tipo=tipo_cruce)
        hijo1, hijo2 = cruzar(antecesor1, antecesor2, tipo=tipo_cruce)
        nuevas_sols.append(hijo1)
        if len(nuevas_sols) < num_sols_total:
            nuevas_sols.append(hijo2)

    for sol_act in nuevas_sols:
        if mut_prob > random.random():
            pos_to_mutate = randint(0, len(sol_act) - 1)
            sol_act[pos_to_mutate] = 1 - sol_act[pos_to_mutate]

    return nuevas_sols


In [ ]:
random.seed(1)

caso = trainX[12][:int(tamMatriz)]
N = 20  # Número de vértices

num_sols_total = 20
iters = 100

poblacion = generar_pobl(num_sols_total, N)

valores_medios = [media_fitness(poblacion, caso)]
valores_min = [min_fitness(poblacion, caso)]
for i in range(iters):
    poblacion = op_genetico(poblacion, caso, num_sols_total, tipo_cruce='dos_puntos')
    valores_medios.append(media_fitness(poblacion, caso))
    valores_min.append(min_fitness(poblacion, caso))

print("Valores medios de fitness a lo largo de las iteraciones:", valores_medios)
print("Valores mínimos de fitness a lo largo de las iteraciones:", valores_min)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.title(r"Media del Valor $Fitness$ de la Población de Soluciones sin Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ medio",fontsize = 15)
plt.ylim( 5 , 25)
plt.plot(range(iters+1), valores_medios,'b',label=r'$Fitness$ medio', linewidth=2)
plt.legend(fontsize = 15,loc = 'upper right')
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado
plt.show()

plt.figure(figsize=(10, 6))
plt.title(r"Valor $Fitness$ Mínimo de la Población de Soluciones sin Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ mínimo",fontsize = 15)
plt.plot(range(iters+1), valores_min,'b',label=r'$Fitness$ máximo', linewidth=2)



plt.legend(fontsize = 15,loc = 'upper right')
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Algoritmo genético con Red Neuronal

In [ ]:
random.seed(1)

caso = trainX[12][:int(tamMatriz)]
N = 20  # Número de vértices

num_sols_total = 20
iters = 100


eps = 0.1
atasco = 0
red_prob = 0.2

poblacion = generar_pobl(num_sols_total, N)

valores_medios = [media_fitness(poblacion, caso)]
valores_min = [min_fitness(poblacion, caso)]
iters_modelo = []
for i in range(iters):
    poblacion = op_genetico(poblacion, caso, num_sols_total, tipo_cruce='dos_puntos')
    valores_medios.append(media_fitness(poblacion, caso))
    valores_min.append(min_fitness(poblacion, caso))
    if valores_min[-1] < (valores_medios[-2]*(1+eps)) :
        atasco += 1
        if atasco >= 10:
          iters_modelo.append(i)
          atasco = 0
          for sol_act in poblacion:
           if red_prob > random.random():
              y_pred = final_model.predict(np.array([concatenar_solucion_instancia(caso, sol_act)]),verbose=0)
              umbral = 0.5
              y_pred_binary = (y_pred >= umbral).astype(int)
              sol_fact = transformar_soluciones_no_factibles(caso, y_pred_binary[0])
              sol_act = sol_fact.tolist()
    else: atasco = 0

print("Valores medios de fitness a lo largo de las iteraciones:", valores_medios)
print("Valores mínimos de fitness a lo largo de las iteraciones:", valores_min)

In [ ]:
plt.figure(figsize=(10, 6))
plt.title(r"Media del Valor $Fitness$ de la Población de Soluciones con Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ medio",fontsize = 15)
plt.plot(range(iters+1), valores_medios,'g',label=r'$Fitness$ Medio', linewidth=2)
plt.scatter(iters_modelo, [valores_medios[i] for i in iters_modelo], color='purple', label='Llamada a la Red Neuronal', s=100, zorder=5)
plt.ylim( 5 , 25)
plt.legend(fontsize = 15,loc = 'upper right')
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()



plt.figure(figsize=(10, 6))
plt.title(r"Valor $Fitness$ Mínimo de la Población de Soluciones con Red Neuronal",fontsize = 15)
plt.xlabel("Iteración",fontsize = 15)
plt.ylabel(r"Valor $fitness$ mínimo",fontsize = 15)
plt.plot(range(iters+1), valores_min,'g',label=r'$Fitness$ mínimo', linewidth=2)
plt.scatter(iters_modelo, [valores_min[i] for i in iters_modelo], color='purple', label='Llamada a la Red Neuronal', s=100, zorder=5)

plt.legend(fontsize = 15, loc = 'upper right')
plt.grid(visible=True,linewidth=0.2) # poner la cuadricula con el grosor del resultado dado

plt.show()

### Ejemplos de Soluciones Generadas

Solución inicial

In [ ]:
sol_ini = transformar_soluciones_no_factibles(caso,trainX[12][-20:])
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(caso,sol_ini)]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(caso, y_pred_binary[0])
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness:", int(valor_fitness(sol_ini, caso)))
print("Solución generada",sol_act, "Valor fitness:", int(valor_fitness(sol_act, caso)))

Solución etiqueta

In [ ]:
sol_ini = transformar_soluciones_no_factibles(caso,trainY[12])
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(caso,sol_ini)]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(caso, y_pred_binary[0])
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness:", int(valor_fitness(sol_ini, caso)))
print("Solución generada",sol_act, "Valor fitness:", int(valor_fitness(sol_act, caso)))

Otra solución distinta

In [ ]:
sol_ini = transformar_soluciones_no_factibles(caso,trainY[7])
y_pred = final_model.predict(np.array([concatenar_solucion_instancia(caso,sol_ini)]),verbose=0)
umbral = 0.5
y_pred_binary = (y_pred >= umbral).astype(int)
sol_fact = transformar_soluciones_no_factibles(caso, y_pred_binary[0])
sol_act = sol_fact.tolist()
print("Solución Inicial:",sol_ini, "Valor fitness:", int(valor_fitness(sol_ini, caso)))
print("Solución generada",sol_act, "Valor fitness:", int(valor_fitness(sol_act, caso)))